In [1]:
import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.nn.functional as F
from torchvision import transforms, models

from PIL import Image, ImageFile
import pickle

import pandas as pd
import numpy as np
import random
#import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import os

In [2]:
def set_seed(seed):
    os.environ["CUBLAS_WORKSPACE_CONFIG"]=":4096:2"
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.backends.cudnn.deterministic = True

set_seed(42)

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [2]:
# Labels:
# White = 0
# Black = 1
train_df = pd.read_csv('train_labels_modificado.csv', index_col=0)
val_df = pd.read_csv('val_labels_modificado.csv', index_col=0)

In [3]:
train_df['race'].value_counts()

race
0    16527
1    12233
Name: count, dtype: int64

In [4]:
val_df['race'].value_counts()

race
0    2085
1    1556
Name: count, dtype: int64

In [8]:
class CustomDataset(Dataset):
    def __init__(self, data, transform):
        self.data = data
        self.transform = transform
    
    def __getitem__(self, index):
        image_path = self.data['file'][index]
        label = self.data['race'][index]
        image = Image.open(image_path)
        image = image.convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, label
    
    def __len__(self):
        return self.data.shape[0]

In [9]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

In [10]:
num_epochs = 20
lr = 0.1
batch_size = 256

In [11]:
custom_train = CustomDataset(train_df, transform=transform)
custom_val = CustomDataset(val_df, transform=transform)

In [12]:
train_loader = DataLoader(custom_train, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(custom_val, batch_size=batch_size)

In [13]:
weights = pickle.load(open('resnet50_scratch_weight.pkl', 'rb'))
model = models.resnet50(weights=weights)

c:\Users\DELL\anaconda3\envs\CUDA\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [14]:
for param in model.parameters():
    param.requires_grad = False

# for param in model.layer4.parameters():
#      param.requires_grad = True

model.fc = nn.Linear(2048, 2)
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [15]:
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
scheduler = lr_scheduler.StepLR(optimizer, step_size=6, gamma=0.2)

model.train()
for epoch in range(num_epochs):
    running_loss = 0
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        output = model(images)
        loss = criterion(output.squeeze(), labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    scheduler.step()
    print(f'Epoch: {epoch + 1} Loss: {running_loss/len(train_loader)}')

Epoch: 1 Loss: 9.25318007764563
Epoch: 2 Loss: 2.0087585048337955
Epoch: 3 Loss: 1.6545157622447055
Epoch: 4 Loss: 3.0318690084778104
Epoch: 5 Loss: 1.5577848980912066
Epoch: 6 Loss: 3.0734641615268403
Epoch: 7 Loss: 1.8818956597716407
Epoch: 8 Loss: 0.7391824337233485
Epoch: 9 Loss: 0.5637412664637101
Epoch: 10 Loss: 0.5508394225508766
Epoch: 11 Loss: 0.5313848204317346
Epoch: 12 Loss: 0.4630978481959453
Epoch: 13 Loss: 0.4124915304437148
Epoch: 14 Loss: 0.41696267739861415
Epoch: 15 Loss: 0.40316211702549354
Epoch: 16 Loss: 0.4022051752668566
Epoch: 17 Loss: 0.4016549779778033
Epoch: 18 Loss: 0.39919004767341953
Epoch: 19 Loss: 0.3863274816390687
Epoch: 20 Loss: 0.38484142470148813


In [16]:
# Teste

correct = 0
total = 0

model.eval()
with torch.no_grad():
    for images, labels in val_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {100 * correct // total} %')

Accuracy: 86 %


In [17]:
# Matriz de confusão

y_pred = []
y_true = []
classes = ['White', 'Black']

model.eval()
model.cpu()
with torch.no_grad():
    for images, labels in val_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        predicted = predicted.cpu()
        y_pred.extend(predicted.numpy())
        y_true.extend(labels.numpy())

    y_pred = np.array(y_pred)
    y_true = np.array(y_true)

    cm = confusion_matrix(y_true, y_pred)

    print('\t\tPredicted Labels\n')

    print(f"True Labels White {cm[0]}")
    print(f"\t       Black {cm[1]}") 
    print('\t\t   White Black')

		Predicted Labels

True Labels White [1856  229]
	       Black [ 255 1301]
		   White Black
